<a href="https://colab.research.google.com/github/joshhawkins119/UFC_Fight_Outcome_Predictor/blob/main/UFC_Streamlined_Stacked_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


General Imports

In [53]:
# Data Prep Imports 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import sys
import datetime 
import seaborn as sns
import datetime
from pytz import utc, timezone
import time
import re
from functools import reduce
import holidays
from dateutil.easter import *
from math import sqrt
import joblib


Supervised ML Model Imports

In [54]:
# ML Modeling Imports

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, accuracy_score, make_scorer

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold, train_test_split, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Ridge, ElasticNet, SGDRegressor, Lasso, RidgeCV, LassoCV, ElasticNetCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


Read in Data

In [55]:

ufc_training_df = pd.read_csv('/content/gdrive/MyDrive/Supervised ML/UFC Fight Predictor/ufc_clean_data_diffs.csv', index_col=0, low_memory=False)
ufc_training_df

,Date,Fighter,Opponent,Weight_Class,Win,Date_of_Last_Fight,Male,Female,DOB,STANCE,Opponent_Weight_Class,Opponent_Date_of_Last_Fight,Opponent_DOB,Opponent_STANCE,Year,Month,Diff_Months_Since_Last_Fight,Diff_Height,Diff_Reach,Diff_Weight,Diff_Round_AOLF,Diff_Won_Last_Fight_AOLF,Diff_Won_Fight_Before_Last_AOLF,Diff_no_ufc_fights_AOLF,Diff_win_total_AOLF,Diff_win_percentage_AOLF,Diff_wins_last_3_AOLF,Diff_wins_last_3_percentage_AOLF,Diff_Won_Last_Two_Fights_AOLF,Diff_Winning_Streak_AOLF,Diff_Fighter_Pass_avg_AOLF,Diff_Fighter_STR_avg_AOLF,Diff_Fighter_Sub_avg_AOLF,Diff_Fighter_TD_avg_AOLF,Diff_Round_avg_AOLF,Diff_DQ_AOLF,Diff_KO/TKO_AOLF,Diff_M-DEC_AOLF,Diff_S-DEC_AOLF,Diff_SUB_AOLF,Diff_U-DEC_AOLF,Diff_DQ_total_AOLF,Diff_DQ_perc_AOLF,Diff_KO/TKO_total_AOLF,Diff_KO/TKO_perc_AOLF,Diff_M-DEC_total_AOLF,Diff_M-DEC_perc_AOLF,Diff_S-DEC_total_AOLF,Diff_S-DEC_perc_AOLF,Diff_SUB_total_AOLF,Diff_SUB_perc_AOLF,Diff_U-DEC_total_AOLF,Diff_U-DEC_perc_AOLF,Diff_Fight_Time_in_Sec_AOLF,Diff_Fight_Time_in_Sec_avg_AOLF,Diff_Fight_Time_in_Min_AOLF,Diff_Fight_Time_in_Min_avg_AOLF,Diff_Min_in_Octogon_Total_AOLF,Diff_avg_level_of_opponents_faced_AOLF,Diff_age
13,1995-12-16,Dan Severn,Oleg Taktarov,Open Weight,1,1995-07-14,1,0,1958-06-08,Southpaw,Open Weight,1995-07-14,1967-08-26,Orthodox,1995,12,0.00,0.2,71.75,40.0,0.0,0.0,0.0,2.0,1.0,-0.09,-1.0,-0.33,-1.0,-1.0,-0.83,0.00,1.00,1.91,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.06,0.0,0.0,0.0,0.00,2.0,0.06,0.0,0.00,125.0,-52.71,2.08,-0.88,4.09,0.090000,9.22
14,1995-12-16,Oleg Taktarov,Dan Severn,Open Weight,0,1995-07-14,1,0,1967-08-26,Orthodox,Open Weight,1995-07-14,1958-06-08,Southpaw,1995,12,0.00,-0.2,-71.75,-40.0,0.0,0.0,0.0,-2.0,-1.0,0.09,1.0,0.33,1.0,1.0,0.83,0.00,-1.00,-1.91,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.0,0.00,-2.0,-0.06,0.0,0.00,-125.0,52.71,-2.08,0.88,-4.09,-0.090000,-9.22
16,1995-12-16,Marco Ruas,Oleg Taktarov,Open Weight,0,1995-12-16,1,0,1961-01-23,Orthodox,Open Weight,1995-12-16,1967-08-26,Orthodox,1995,12,0.00,0.1,0.10,0.0,-1.0,0.0,0.0,-2.0,0.0,0.33,1.0,0.33,1.0,1.0,0.17,0.25,0.50,9.25,-0.17,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.00,-1.0,0.08,-1.0,-0.17,-21.0,190.67,-0.35,3.18,3.18,-0.185000,6.59
17,1995-12-16,Oleg Taktarov,Marco Ruas,Open Weight,1,1995-12-16,1,0,1967-08-26,Orthodox,Open Weight,1995-12-16,1961-01-23,Orthodox,1995,12,0.00,-0.1,-0.10,0.0,1.0,0.0,0.0,2.0,0.0,-0.33,-1.0,-0.33,-1.0,-1.0,-0.17,-0.25,-0.50,-9.25,0.17,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.08,0.0,0.0,0.0,0.00,1.0,-0.08,1.0,0.17,21.0,-190.67,0.35,-3.18,-3.18,0.185000,-6.59
22,1996-05-17,Dan Severn,Ken Shamrock,Open Weight,1,1995-12-16,1,0,1958-06-08,Southpaw,Open Weight,1996-02-16,1964-02-11,Orthodox,1996,5,2.04,0.1,5.75,45.0,0.0,0.0,0.0,6.0,4.0,-0.20,0.0,0.00,0.0,0.0,-0.65,0.00,0.15,6.50,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.15,0.0,0.0,0.0,0.00,4.0,-0.05,2.0,0.20,-203.0,69.85,-3.38,1.16,35.67,0.100000,5.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6793,2021-07-24,Cory Sandhagen,TJ Dillashaw,Bantamweight,0,2021-02-06,1,0,1992-04-20,Switch,Bantamweight,2019-01-19,1986-02-07,Orthodox,2021,7,-24.56,0.4,3.00,0.0,0.0,0.0,-1.0,-8.0,-5.0,0.13,0.0,0.00,1.0,0.0,-0.50,-0.06,-0.81,-14.56,-0.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-0.06,0.0,0.0,-1.0,0.00,1.0,0.19,-3.0,-0.13,-4.0,-35.06,-0.06,-0.58,-30.64,-0.113788,-6.20
6795,2021-07-24,Raulian Paiva,Kyler Phillips,Bantamweight,1,2020-07-11,1,0,1995-10-17,Orthodox,Bantamweight,2021-03-06,1995-06-12,Orthodox,2021,7,7.80,0.0,-3.00,-10.0,0.0,0.0,-1.0,1.0,-1.0,-0.50,-1.0,-0.33,0.0,-1.0,-0.08,0.25,-2.33,-16.00,-0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.17,0.0,0.0,1.0,0.25,0.0,0.00,-1.0,-0.42,0.0,49.83,0.00,0.83,6.90,0.500000,-0.34
6797,2021-07-24,Julio Arce,Andre Ewell,Bantamweight,1,2019-11-02,1,0,1989-10-27,Southpaw,Bantamweight,2021-

Base Line Accuracy Tests

In [56]:
# BASELINE MODEL 1. All win or All Lose - 50% Accurate


# BASELINE MODEL 2. 'Always pick the younger fighter' 

# ufc_training_df['base_pred'] = np.where(ufc_training_df['Fighter_Age'] < ufc_training_df['Opponent_Age'], 1, 0)
ufc_training_df['base_pred'] = np.where(ufc_training_df['Diff_age'] < 0, 1, 0)

# Train/Test Split
X = ufc_training_df
y = ufc_training_df['Win']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 314)

# Accuracy Check
base_line_2 = accuracy_score(y_val, X_val['base_pred'])

base_line_results = {
    'base line 1 accuracy' : '50%',
    'base line 2 accuracy' : round(100*base_line_2, 2)
}

print(base_line_results)

{'base line 1 accuracy': '50%', 'base line 2 accuracy': 54.55}


Choose Features, Establish Label 

In [57]:
#Initialize features
non_feature_list = ['Career_Stage', 'Male', 'Female', 'base_pred', 'Diff_Weight', 'Month', 'AGE',  'Win', 'Date', 'Date_of_Last_Fight', 'Opponent_Date_of_Last_Fight', 'Fighter', 'Opponent', 'DOB', 
                    'Opponent_DOB', 'Opponent_STANCE', 'STANCE', 'Weight_Class', 'Opponent_STANCE', 'Opponent_Weight_Class', 
                    'base_pred', 'Diff_wins_last_5_AOLF', 'Diff_Fight_Time_in_Sec_AOLF', 'Diff_Fight_Time_in_Min_AOLF', 'Diff_Fight_Time_in_Min_avg_AOLF']  #'Diff_Fight_Time_in_Sec_avg_AOLF'
feature_list = [i for i in ufc_training_df.columns.tolist() if i not in non_feature_list] 


redundant_or_non_diff_friendly = ['Diff_win_total_AOLF', 'Diff_DQ_total_AOLF', 'Diff_M-DEC_total_AOLF', 'Diff_S-DEC_total_AOLF', 
                                  'Diff_SUB_total_AOLF', 'Diff_KO/TKO_total_AOLF', 'Diff_U-DEC_total_AOLF']
feature_list = [i for i in feature_list if i not in redundant_or_non_diff_friendly] 


ufc_training_df['Win'] = ufc_training_df['Win'].astype(int)
recent_years = ufc_training_df['Year']>2004
ufc_training_df = ufc_training_df[recent_years]

personal_bias_features = ['Diff_Months_Since_Last_Fight', 'Diff_wins_last_3_AOLF', 'Diff_Reach', 'Diff_age', 
                          'Diff_Fighter_Pass_avg_AOLF', 'Diff_Fighter_STR_avg_AOLF', 'Diff_Fighter_Sub_avg_AOLF',
                          'Diff_Fighter_TD_avg_AOLF', 'Diff_Round_avg_AOLF', 'Diff_avg_level_of_opponents_faced_AOLF',
                          'Diff_no_ufc_fights_AOLF', 'Diff_KO/TKO_AOLF']

ufc_training_df

,Date,Fighter,Opponent,Weight_Class,Win,Date_of_Last_Fight,Male,Female,DOB,STANCE,Opponent_Weight_Class,Opponent_Date_of_Last_Fight,Opponent_DOB,Opponent_STANCE,Year,Month,Diff_Months_Since_Last_Fight,Diff_Height,Diff_Reach,Diff_Weight,Diff_Round_AOLF,Diff_Won_Last_Fight_AOLF,Diff_Won_Fight_Before_Last_AOLF,Diff_no_ufc_fights_AOLF,Diff_win_total_AOLF,Diff_win_percentage_AOLF,Diff_wins_last_3_AOLF,Diff_wins_last_3_percentage_AOLF,Diff_Won_Last_Two_Fights_AOLF,Diff_Winning_Streak_AOLF,Diff_Fighter_Pass_avg_AOLF,Diff_Fighter_STR_avg_AOLF,Diff_Fighter_Sub_avg_AOLF,Diff_Fighter_TD_avg_AOLF,Diff_Round_avg_AOLF,Diff_DQ_AOLF,Diff_KO/TKO_AOLF,Diff_M-DEC_AOLF,Diff_S-DEC_AOLF,Diff_SUB_AOLF,Diff_U-DEC_AOLF,Diff_DQ_total_AOLF,Diff_DQ_perc_AOLF,Diff_KO/TKO_total_AOLF,Diff_KO/TKO_perc_AOLF,Diff_M-DEC_total_AOLF,Diff_M-DEC_perc_AOLF,Diff_S-DEC_total_AOLF,Diff_S-DEC_perc_AOLF,Diff_SUB_total_AOLF,Diff_SUB_perc_AOLF,Diff_U-DEC_total_AOLF,Diff_U-DEC_perc_AOLF,Diff_Fight_Time_in_Sec_AOLF,Diff_Fight_Time_in_Sec_avg_AOLF,Diff_Fight_Time_in_Min_AOLF,Diff_Fight_Time_in_Min_avg_AOLF,Diff_Min_in_Octogon_Total_AOLF,Diff_avg_level_of_opponents_faced_AOLF,Diff_age,base_pred
232,2005-02-05,Tito Ortiz,Vitor Belfort,Light Heavyweight,1,2004-10-22,1,0,1975-01-23,Orthodox,Light Heavyweight,2004-08-21,1977-04-01,Southpaw,2005,2,-2.03,0.2,0.0,20.0,0.0,-1.0,-1.0,4.0,3.0,0.01,-1.0,-0.34,0.0,0.0,-0.29,-0.33,2.11,19.80,1.10,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.23,0.0,0.0,0.0,0.00,1.0,0.04,3.0,0.19,0.0,52.47,0.00,0.87,23.37,0.090714,2.19,0
233,2005-02-05,Vitor Belfort,Tito Ortiz,Light Heavyweight,0,2004-08-21,1,0,1977-04-01,Southpaw,Light Heavyweight,2004-10-22,1975-01-23,Orthodox,2005,2,2.03,-0.2,0.0,-20.0,0.0,1.0,1.0,-4.0,-3.0,-0.01,1.0,0.34,0.0,0.0,0.29,0.33,-2.11,-19.80,-1.10,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.23,0.0,0.0,0.0,0.00,-1.0,-0.04,-3.0,-0.19,0.0,-52.47,0.00,-0.87,-23.37,-0.090714,-2.19,1
234,2005-02-05,Chris Lytle,Karo Parisyan,Welterweight,0,2004-08-21,1,0,1974-08-18,Orthodox,Welterweight,2004-08-21,1982-08-28,Orthodox,2005,2,0.00,0.1,-4.0,0.0,-1.0,1.0,-1.0,1.0,0.0,-0.17,0.0,0.00,1.0,0.0,-0.42,0.25,-3.25,7.58,-0.08,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,-1.0,-0.33,1.0,0.17,1.0,0.17,-223.0,-79.33,-3.72,-1.32,-0.67,0.170000,8.03,0
235,2005-02-05,Andrei Arlovski,Tim Sylvia,Heavyweight,1,2004-04-02,1,0,1979-02-04,Orthodox,Heavyweight,2004-06-19,1976-03-05,Orthodox,2005,2,2.56,-0.5,-3.0,-25.0,1.0,0.0,0.0,2.0,1.0,-0.08,1.0,0.33,1.0,1.0,0.33,0.17,-0.08,-19.67,0.58,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,0.08,0.0,0.0,0.0,0.00,0.0,-0.08,0.0,0.00,25.0,-28.17,0.42,-0.47,0.98,-0.020000,-2.92,1
237,2005-02-05,Karo Parisyan,Chris Lytle,Welterweight,1,2004-08-21,1,0,1982-08-28,Orthodox,Welterweight,2004-08-21,1974-08-18,Orthodox,2005,2,0.00,-0.1,4.0,0.0,1.0,-1.0,1.0,-1.0,0.0,0.17,0.0,0.00,-1.0,0.0,0.42,-0.25,3.25,-7.58,0.08,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1.0,0.33,-1.0,-0.17,-1.0,-0.17,223.0,79.33,3.72,1.32,0.67,-0.170000,-8.03,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6793,2021-07-24,Cory Sandhagen,TJ Dillashaw,Bantamweight,0,2021-02-06,1,0,1992-04-20,Switch,Bantamweight,2019-01-19,1986-02-07,Orthodox,2021,7,-24.56,0.4,3.0,0.0,0.0,0.0,-1.0,-8.0,-5.0,0.13,0.0,0.00,1.0,0.0,-0.50,-0.06,-0.81,-14.56,-0.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-0.06,0.0,0.0,-1.0,0.00,1.0,0.19,-3.0,-0.13,-4.0,-35.06,-0.06,-0.58,-30.64,-0.113788,-6.20,1
6795,2021-07-24,Raulian Paiva,Kyler Phillips,Bantamweight,1,2020-07-11,1,0,1995-10-17,Orthodox,Bantamweight,2021-03-06,1995-06-12,Orthodox,2021,7,7.80,0.0,-3.0,-10.0,0.0,0.0,-1.0,1.0,-1.0,-0.50,-1.0,-0.33,0.0,-1.0,-0.08,0.25,-2.33,-16.00,-0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.17,0.0,0.0,1.0,0.25,0.0,0.00,-1.0,-0.42,0.0,49.83,0.00,0.83,6.90,0.500000,-0.34,1
6797,2021-07-24,Julio Arce,Andre Ewell,Bantam

Hold Out Set Creation and Single Model Predictor with Permutation Feature Importance


In [58]:

#Separate men and women
mens_ufc = ufc_training_df.loc[ufc_training_df.Male == 1]
womens_ufc = ufc_training_df.loc[ufc_training_df.Female == 1]

#Create Holdouts
most_recent_date = ufc_training_df['Date'].sort_values(ascending=False).unique()[0]
mens_holdout = mens_ufc.loc[mens_ufc.Date == most_recent_date]
mens_train = mens_ufc.loc[~(mens_ufc.Date == most_recent_date)]

womens_holdout = womens_ufc.loc[womens_ufc.Date == most_recent_date]
womens_train = womens_ufc.loc[~(womens_ufc.Date == most_recent_date)]


model = RandomForestClassifier(n_estimators= 1000, max_features='sqrt', max_depth=6, random_state=314, oob_score=True)
#model = MLPClassifier() 
#model = LogisticRegressionCV(max_iter=5000)
# model = XGBClassifier(n_estimators=1000, random_state=314, learning_rate= 0.05, max_depth= 3)
# model = KNeighborsClassifier() 

def predict_fights_score(df, features, target, model=model):
  X = df[features]
  y = df[target]  
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 314)
  model.fit(X_train, y_train)

  #feature imp
  result = permutation_importance(model, X_train, y_train, n_repeats=10, random_state=314, n_jobs=-1)
  perm_df = pd.DataFrame({'cols':X_train.columns, 'imp':result.importances_mean}).sort_values('imp', ascending=False)
  to_keep = perm_df[perm_df.imp>0.005].cols

  #re-train, re-fit
  X_train = X_train[to_keep]
  X_val = X_val[to_keep]
  model = model.fit(X_train, y_train)
  

  return model.score(X_val, y_val) #model


'men prediction accuracy: {}'.format(predict_fights_score(mens_train, feature_list, 'Win')), 'women prediction accuracy: {}'.format(predict_fights_score(womens_train, feature_list, 'Win'))

('men prediction accuracy: 0.5838150289017341',
 'women prediction accuracy: 0.6285714285714286')

Stacked Model Approach

In [59]:

def predict_fights_score_stacked(df, features, target):
  X = df[features]
  y = df[target]  
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 314)

  base_models = [('random_forest', RandomForestClassifier(random_state=314, oob_score=True)),
               ('lgr', LogisticRegression(random_state=314)),
               ('knn', KNeighborsClassifier()),
               ('mlp', MLPClassifier(random_state=314)),
               ('xgb', XGBClassifier(random_state=314, oob_score=True))]
  #meta_model = RandomForestClassifier(n_estimators= 1000, max_features='sqrt', max_depth=6, random_state=314, oob_score=True)
  meta_model = LogisticRegressionCV(max_iter=5000, random_state=314)
  stacking_model = StackingClassifier(estimators=base_models, 
                                      final_estimator=meta_model, 
                                      passthrough=True, 
                                      cv=5)
  # Scale
  numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
  preprocessor = ColumnTransformer(transformers=[
          ('num', numeric_transformer, features)])

  # Create pipeline 
  pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('stk', stacking_model)])


  pipeline.fit(X_train, y_train)

  return pipeline, pipeline.score(X_val, y_val)


 #Non Selection Version
womens_model, womens_score = predict_fights_score_stacked(womens_train, feature_list, 'Win')
mens_model, mens_score = predict_fights_score_stacked(mens_train, feature_list, 'Win')


'men prediction accuracy: {}'.format(mens_score), 'women prediction accuracy: {}'.format(womens_score)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


('men prediction accuracy: 0.576878612716763',
 'women prediction accuracy: 0.7571428571428571')

Test on HoldOut Set

In [60]:
# Testing against all predictions 

test = mens_holdout.copy()
if mens_holdout.shape[0] > 0:
  hold_out_mens_score = mens_model.score(test[feature_list], test.Win)

test2 = womens_holdout.copy()
if womens_holdout.shape[0] > 0:
  hold_out_womens_score = womens_model.score(test2[feature_list], test2.Win)

if mens_holdout.shape[0] == 0:
  print('hold out women prediction accuracy: {}'.format(hold_out_womens_score))
if womens_holdout.shape[0] == 0:
  print('hold out men prediction accuracy: {}'.format(hold_out_mens_score))
if (mens_holdout.shape[0] > 0)&(womens_holdout.shape[0] > 0):
  print('hold out men prediction accuracy: {}'.format(hold_out_mens_score), 'hold out women prediction accuracy: {}'.format(hold_out_womens_score))
else:
  pass
 

hold out men prediction accuracy: 0.125


In [61]:
# Testing only on rows with the highest probability

mens_test = mens_holdout.copy()
if mens_holdout.shape[0] > 0:
  mens_test['win_proba'] = mens_model.predict_proba(mens_test[feature_list])[:, 1]

womens_test = womens_holdout.copy()
if womens_holdout.shape[0] > 0:
  womens_test['win_proba'] = womens_model.predict_proba(womens_test[feature_list])[:, 1]

def high_prob_score(df):
  results_df = pd.DataFrame()
  for i in df.Fighter.unique().tolist():
    temp_df = df.loc[df.isin([i]).any(axis=1)]
    temp_df = temp_df[temp_df.win_proba == temp_df.win_proba.max()]
    results_df = pd.concat([results_df, temp_df])
    results_df.drop_duplicates(inplace=True)
  return results_df

full_results = pd.concat([high_prob_score(mens_test), high_prob_score(womens_test)])
full_results['prediction'] = full_results['win_proba']
full_results

,Date,Fighter,Opponent,Weight_Class,Win,Date_of_Last_Fight,Male,Female,DOB,STANCE,Opponent_Weight_Class,Opponent_Date_of_Last_Fight,Opponent_DOB,Opponent_STANCE,Year,Month,Diff_Months_Since_Last_Fight,Diff_Height,Diff_Reach,Diff_Weight,Diff_Round_AOLF,Diff_Won_Last_Fight_AOLF,Diff_Won_Fight_Before_Last_AOLF,Diff_no_ufc_fights_AOLF,Diff_win_total_AOLF,Diff_win_percentage_AOLF,Diff_wins_last_3_AOLF,Diff_wins_last_3_percentage_AOLF,Diff_Won_Last_Two_Fights_AOLF,Diff_Winning_Streak_AOLF,Diff_Fighter_Pass_avg_AOLF,Diff_Fighter_STR_avg_AOLF,Diff_Fighter_Sub_avg_AOLF,Diff_Fighter_TD_avg_AOLF,Diff_Round_avg_AOLF,Diff_DQ_AOLF,Diff_KO/TKO_AOLF,Diff_M-DEC_AOLF,Diff_S-DEC_AOLF,Diff_SUB_AOLF,Diff_U-DEC_AOLF,Diff_DQ_total_AOLF,Diff_DQ_perc_AOLF,Diff_KO/TKO_total_AOLF,Diff_KO/TKO_perc_AOLF,Diff_M-DEC_total_AOLF,Diff_M-DEC_perc_AOLF,Diff_S-DEC_total_AOLF,Diff_S-DEC_perc_AOLF,Diff_SUB_total_AOLF,Diff_SUB_perc_AOLF,Diff_U-DEC_total_AOLF,Diff_U-DEC_perc_AOLF,Diff_Fight_Time_in_Sec_AOLF,Diff_Fight_Time_in_Sec_avg_AOLF,Diff_Fight_Time_in_Min_AOLF,Diff_Fight_Time_in_Min_avg_AOLF,Diff_Min_in_Octogon_Total_AOLF,Diff_avg_level_of_opponents_faced_AOLF,Diff_age,base_pred,win_proba,prediction
6800,2021-07-24,Darrick Minner,Darren Elkins,Featherweight,0,2021-02-20,1,0,1990-04-28,Orthodox,Featherweight,2020-11-07,1984-05-16,Orthodox,2021,7,-3.44,-0.2,-2.0,0.0,0.0,1.0,0.0,-20.0,-13.0,0.02,1.0,0.34,1.0,0.0,1.04,0.24,-0.50,-20.97,-0.61,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-5.0,-0.22,0.0,0.0,-2.0,-0.09,-1.0,0.54,-12.0,-0.24,158.0,-92.22,2.63,-1.54,-85.35,-0.064706,-5.95,1,0.550227,0.550227
6790,2021-07-24,Kyler Phillips,Raulian Paiva,Bantamweight,0,2021-03-06,1,0,1995-06-12,Orthodox,Bantamweight,2020-07-11,1995-10-17,Orthodox,2021,7,-7.80,0.0,3.0,10.0,0.0,0.0,1.0,-1.0,1.0,0.50,1.0,0.33,0.0,1.0,0.08,-0.25,2.33,16.00,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-0.17,0.0,0.0,-1.0,-0.25,0.0,0.00,1.0,0.42,0.0,-49.83,0.00,-0.83,-6.90,-0.500000,0.34,0,0.560589,0.560589
6797,2021-07-24,Julio Arce,Andre Ewell,Bantamweight,1,2019-11-02,1,0,1989-10-27,Southpaw,Bantamweight,2021-02-13,1988-01-21,Southpaw,2021,7,15.38,-0.1,-5.0,10.0,0.0,0.0,-1.0,-2.0,-1.0,0.03,-1.0,-0.34,0.0,0.0,0.46,0.06,0.31,-11.51,0.00,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.06,0.0,0.0,-1.0,-0.03,0.0,0.06,-1.0,-0.09,0.0,-41.63,0.00,-0.69,-12.74,-0.149167,-1.77,1,0.483044,0.483044
6793,2021-07-24,Cory Sandhagen,TJ Dillashaw,Bantamweight,0,2021-02-06,1,0,1992-04-20,Switch,Bantamweight,2019-01-19,1986-02-07,Orthodox,2021,7,-24.56,0.4,3.0,0.0,0.0,0.0,-1.0,-8.0,-5.0,0.13,0.0,0.00,1.0,0.0,-0.50,-0.06,-0.81,-14.56,-0.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-0.06,0.0,0.0,-1.0,0.00,1.0,0.19,-3.0,-0.13,-4.0,-35.06,-0.06,-0.58,-30.64,-0.113788,-6.20,1,0.676162,0.676162


RETRAIN ON ALL DATA

In [62]:
final_womens_model, final_womens_score = predict_fights_score_stacked(womens_ufc, feature_list, 'Win')
final_mens_model, final_mens_score = predict_fights_score_stacked(mens_ufc, feature_list, 'Win')

'men prediction accuracy: {}'.format(final_mens_score), 'women prediction accuracy: {}'.format(final_womens_score)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


('men prediction accuracy: 0.6043829296424452',
 'women prediction accuracy: 0.7571428571428571')

SAVE MODELS


In [63]:
# Save mens model
mens_filename = '/content/gdrive/MyDrive/Supervised ML/UFC Fight Predictor/mens_stack_model.pkl'
joblib.dump(final_mens_model, mens_filename)

['/content/gdrive/MyDrive/Supervised ML/UFC Fight Predictor/mens_stack_model.pkl']

In [64]:
# Save womens model
womens_filename = '/content/gdrive/MyDrive/Supervised ML/UFC Fight Predictor/womens_stack_model.pkl'
joblib.dump(final_womens_model, womens_filename)

['/content/gdrive/MyDrive/Supervised ML/UFC Fight Predictor/womens_stack_model.pkl']